In [23]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from collections import OrderedDict
from math import log, sqrt
from bokeh.layouts import column, row
from bokeh.models import HoverTool, ColumnDataSource
output_notebook()

TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

data = pd.read_csv('Wholesale customers data-missing.csv')
columns = data.columns

region_color = OrderedDict([
    ("Lisbon",   "#0d3362"),
    ("Oporto", "#c64737"),
    ("Other Region",     "black"  ),
])

dwarf_constant = 10
width = 700
height = 700
inner_radius = 90
outer_radius = 300 - 10
delta = outer_radius - inner_radius

minr = sqrt(log(0.01*1E4))
maxr = sqrt(log(2000 * 1E4))
a = delta / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic, max_vals):
    return inner_radius + (delta*mic/max_vals)

def circle_renderer(df, categorical_column, columns_to_drop):
    new_df = df.drop(columns_to_drop, axis=1)
    grouped = new_df.groupby(categorical_column)
    new_df = grouped.aggregate('mean')
    big_angle = 2.0 * np.pi / (len(new_df.columns))
    small_angle = big_angle / 7
    p = figure(tools=TOOLS, plot_width=width, plot_height=height, title="",
        x_axis_type=None, y_axis_type=None,
        x_range=(-420, 420), y_range=(-420, 420),
        min_border=0, outline_line_color="black",
        background_fill_color="#f0e1d2", border_fill_color="#f0e1d2",
        toolbar_sticky=False)

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    # annular wedges
    angles = np.pi/2 - big_angle/2 - np.arange(0, len(new_df.columns))*big_angle
    colors = ["#aeaeb8" for gram in new_df.columns]
    p.annular_wedge(
        0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
    )

    # small wedges
    max_vals = new_df.max()
    p.annular_wedge(0, 0, inner_radius, rad(new_df.loc[1], max_vals),
                    -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                    color=region_color['Lisbon'])
    p.annular_wedge(0, 0, inner_radius, rad(new_df.loc[2], max_vals),
                    -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                    color=region_color['Oporto'])
    p.annular_wedge(0, 0, inner_radius, rad(new_df.loc[3], max_vals),
                    -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                    color=region_color['Other Region'])

    # circular axes and lables
    labels = np.power(10.0, np.arange(-3, 4))
    radii = a * np.sqrt(np.log(labels * 1E4)) + b
    p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
    p.text(0, radii[2:], ['$'+str((1/r)*1000) for r in labels[2:]],
           text_font_size="8pt", text_align="center", text_baseline="middle")

    # radial axes
    p.annular_wedge(0, 0, inner_radius, outer_radius,
                    -big_angle+angles, -big_angle+angles, color="black")

    # product labels
    xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
    yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
    label_angle=np.array(-big_angle/2+angles)
    label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
    p.text(xr, yr, new_df.columns, angle=label_angle,
           text_font_size="9pt", text_align="center", text_baseline="middle")

    p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
           color=list(region_color.values()))
    p.text([-15, -15, -15], [18, 0, -18], text=list(region_color),
           text_font_size="9pt", text_align="left", text_baseline="middle")
    return p

def scatter_plots(df, column1, column2, radius_column, take_log=False):
    if take_log:
        x = np.log(df[column1])
        y = np.log(df[column2])
        radii = np.log(df[radius_column])/(dwarf_constant*x.mean())
    else:
        x = df[column1]
        y = df[column2]
        radii = df[radius_column]/(dwarf_constant*x.mean())
    colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(x, y)]
    p = figure(tools=TOOLS, plot_width=width, plot_height=height, title=column1+" vs "+column2 +" scatter plot (radius of circles="+radius_column+")")
    p.scatter(x, y, radius=radii,
          fill_color=colors, fill_alpha=0.6, line_color=None)
    p.xaxis.axis_label = column1
    p.yaxis.axis_label = column2
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ])
    p.add_tools(hover)
    return p

Loading BokehJS ...

In [24]:
# approach 1: ignore row with missing value
data1 = data.dropna(axis=0)
p1 = circle_renderer(data1, "Region", "Channel")
p1_scatter = scatter_plots(data1, "Milk", "Fresh", "Grocery", True)
layout = column([p1, p1_scatter])
show(layout)

In [26]:
# approach 2: fill missing value with 1
data2 = data.fillna(1)
p2 = circle_renderer(data2, "Region", "Channel")
p2_scatter = scatter_plots(data2, "Milk", "Fresh", "Grocery", True)
layout = column([p2, p2_scatter])
show(layout)

In [5]:
# approach 3: fill it with mean value
mean = data.mean(axis=0)
data3 = data.fillna(mean)
p3 = circle_renderer(data3, "Region", "Channel")
show(p3)

In [3]:
# approach 4: fill it with EM algorithm
